# Some notes & goals

Xelix previously used a rule-based model to classify emails as "Statements", however, it didn't do that well (like mislabelled things). They are keen to experiment with something else

The best way to add value would be to classify all the emails in the right categories --> however starting with the bank detail changes would be good 

In [1]:
import numpy as np
import pandas as pd
import sqlite3

import matplotlib.pyplot as plt

In [6]:
f_data = "../data/email_extractor.sqlite"
con = sqlite3.connect(f_data)

In [7]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", con)

,type,name,tbl_name,rootpage,sql
0,table,accounts_user,accounts_user,2,CREATE TABLE accounts_user (\n id bigint NO...
1,table,accounts_user_groups,accounts_user_groups,3,CREATE TABLE accounts_user_groups (\n id bi...
2,table,accounts_user_user_permissions,accounts_user_user_permissions,4,CREATE TABLE accounts_user_user_permissions (\...
3,table,auth_group,auth_group,5,CREATE TABLE auth_group (\n id integer NOT ...
4,table,auth_group_permissions,auth_group_permissions,6,CREATE TABLE auth_group_permissions (\n id ...
5,table,auth_permission,auth_permission,7,CREATE TABLE auth_permission (\n id integer...
6,table,django_admin_log,django_admin_log,8,CREATE TABLE django_admin_log (\n id intege...
7,table,django_celery_beat_clockedschedule,django_celery_beat_clockedschedule,9,CREATE TABLE django_celery_beat_clockedschedul...
8,table,django_celery_beat_crontabschedule,django_celery_beat_crontabschedule,10,CREATE TABLE django_celery_beat_crontabschedul...
9,table,django_celery_beat_intervalschedule,django_celery_beat_intervalschedule,11,CREATE TABLE django_celery_beat_intervalschedu...


# There are 279,719 emails in the emails_email table

(Determined by running cell below, which takes a while to run)

In [9]:
# # can count the number of rows in emails_email this way? But it's super slow tho
cursor = con.cursor()
cursor.execute("select count(*) from emails_category")
results = cursor.fetchone()
print(results[0])

48


# Check out what's in the tables

In [4]:
# check out the "id" in the emails_category > need to link this to "category_id" in emails_emailcategory
q = '''
    SELECT * 
    FROM emails_category 
    WHERE (emails_category.slug = 'bank-detail-change')
'''
df = pd.read_sql_query(q,con)
df

,id,created_at,updated_at,name,slug,hidden
0,35,2022-07-25 11:14:09.036276+00,2022-07-25 11:14:09.036299+00,Bank Detail Change,bank-detail-change,f
1,35,2022-07-25 11:14:09.036276+00,2022-07-25 11:14:09.036299+00,Bank Detail Change,bank-detail-change,f


There are two identical slugs and category id's for bank-detail-change: both are 35, they seem to be completely identical, so could probably be merged?

In [5]:
# checking that there is a distinct "bank-detail-change" cell == 35
df = pd.read_sql_query("SELECT DISTINCT category_id FROM emails_emailcategory", con)
df.head(100)

,category_id
0,101
1,1
2,100
3,47
4,48
5,44
6,34
7,35
8,36
9,37


In [6]:
# need to link the "email_id" from emails_emailcategory > to the "id" in the emails_email table
df = pd.read_sql_query("SELECT * FROM emails_email LIMIT 10", con)
df.head(5)

,id,created_at,updated_at,uid,flags,internal_date,bcc,cc,from_address,html_body,message_id,plain_text_body,reply_to,subject,to,eml_file,folder_id
0,306340,2022-07-29 20:00:12.363819+00,2022-07-29 20:00:12.363835+00,8464,\Recent,2022-07-28 22:31:54+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282230.26SMUf7A003811@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/bbbf78a5-ccf2-44dd-af99-2...,3
1,306341,2022-07-29 20:00:12.927732+00,2022-07-29 20:00:12.927749+00,8465,\Recent,2022-07-28 22:32:29+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282231.26SMV1Le003938@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/cb06c705-4beb-44d6-86db-d...,3
2,306342,2022-07-29 20:00:13.208991+00,2022-07-29 20:00:13.209005+00,8466,\Recent,2022-07-28 22:33:27+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282231.26SMVa9Q004201@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/a9c0843e-e907-4013-b742-d...,3
3,306343,2022-07-29 20:00:13.423754+00,2022-07-29 20:00:13.423769+00,8467,\Recent,2022-07-28 22:37:12+00,{},{},no-reply@avery.com,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",<202207282231.26SMV7cM004046@mail.qad.com>,None,{},Avery UK Invoice:,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/8216b595-d169-4521-89ee-3...,3
4,306344,2022-07-29 20:00:13.606982+00,2022-07-29 20:00:13.606995+00,8468,\Recent,2022-07-29 09:41:28+00,{},{},info@phoenixsafe.co.uk,"<!doctype html><html xmlns=""http://www.w3.org/...",<c2e1bcff24eed0dbef1ffd4a0.910c7a73bb.20220729...,Attention: This email originated outside of ou...,{info@phoenixsafe.co.uk},Keep Cash Safe,{purchase.ledger@evo-group.co.uk},email_data/Evo Group/f467154d-fbb3-44cd-96cd-0...,3


In [7]:
# maybe want to know what the attachment is as well?
df = pd.read_sql_query("SELECT * FROM emails_emailattachment LIMIT 10", con)
df.head(5)

,id,created_at,updated_at,content_id,content_type,file,email_id,eml_file,original_filename
0,329935,2022-07-20 15:48:07.211947+00,2022-07-20 15:48:07.211966+00,33F7489332119E4F90DA601FE2F9E282@GBRP265.PROD....,text/html,,167509,None,None
1,329937,2022-07-20 15:48:07.351158+00,2022-07-20 15:48:07.351174+00,FBCB80167B75FE49A35951FB9697A24F@GBRP265.PROD....,text/plain,,167510,None,None
2,329939,2022-07-20 15:48:07.544326+00,2022-07-20 15:48:07.544343+00,5F0AD13A80A15C4DBA0DBE8B35AD8655@GBRP265.PROD....,text/plain,,167511,None,None
3,700737,2022-08-19 13:48:22.371306+00,2022-08-19 13:48:22.371321+00,None,application/pdf,email_attachment/Scripps/4cd82e19-21b5-4625-b8...,343941,,Scanned Document Attached.pdf
4,329941,2022-07-20 15:48:07.730439+00,2022-07-20 15:48:07.730456+00,8EF5410E776C004BA4D49F3013DDAEAC@GBRP265.PROD....,text/plain,,167512,None,None


# Look at the data
How many emails? How many are classified right now?

In [ ]:
# SQL code to extract all bank-detail-change emails
# extract id and slug from emails_category
# link to id == category_id in emails_emailcategory, extract email_id, category_id
# link email_id from emails_emailcategory to id in emails_email

# tried to rename here, didn't really work - get a syntax error from SELECT

# q = '''
#     SELECT id AS category_id FROM emails_category
#     SELECT slug FROM emails_category
#     SELECT email_id AS id FROM emails_emailcategory
#     SELECT category_id from emails_emailcategory
#     SELECT * FROM emails_email
#     WHERE (emails_category.category_id = emails_emailcategory.category_id) AND 
#     (emails_emailcategory.id = emails_email.id) AND emails_emailcategory.category_id IS NOT NULL
# '''

# labelled_emails = pd.read_sql_query(q,con)
# labelled_emails

In [ ]:
# SQL code to extract all bank-detail-change emails
# extract id and slug from emails_category
# link to id == category_id in emails_emailcategory, extract email_id, category_id
# link email_id from emails_emailcategory to id in emails_email

# This syntax includes the emails_emailattachment details too, but it has a row for every attachment (e.g. if there was a
# hyperlink or other type of attachment, so we get so many rows, I just left it out for now)

# q = '''
#     SELECT emails_category.id, emails_category.slug,
#         emails_emailcategory.email_id, emails_emailcategory.category_id,
#         emails_emailattachment.email_id, emails_emailattachment.content_type, emails_emailattachment.original_filename,
#         emails_email.*
#     FROM emails_category, emails_emailcategory, emails_emailattachment, emails_email
#     WHERE (emails_category.id = emails_emailcategory.category_id) AND 
#     (emails_emailcategory.email_id = emails_email.id) AND 
#     (emails_emailattachment.email_id = emails_email.id) AND
#     emails_emailcategory.category_id IS NOT NULL
# '''

# labelled_emails = pd.read_sql_query(q,con)
# labelled_emails

In [ ]:
df = pd.read_sql_query("SELECT * FROM emails_email LIMIT 10", con)
df.head(1)

## The goal of the query below is to try and get the info from the emails_attachment table in here too

This takes so long to run!

In [ ]:
# Define the query
# q = '''
# SELECT e.id, e.internal_date,
#        (SELECT COUNT(*) FROM emails_emailattachment a WHERE a.email_id = e.id) AS attachment_exists,
#        c.id AS category_id, c.slug AS category_slug
# FROM emails_email e, emails_category c, emails_emailcategory ec
# WHERE e.id = ec.email_id AND c.id = ec.category_id AND ec.category_id IS NOT NULL
# '''

# # Execute the query and store the results in a DataFrame
# labelled_emails = pd.read_sql_query(q,con)
# labelled_emails

## Trying potentially faster method, with joins this time

I needed to add backticks around the column 'to' to be able to select it

In the below subquery, the GROUP_CONCAT CASE WHEN line is a condition to only count attachments when the "original_filename" column is not null. Else, everything from hyperlinks to images gets tagged as an attachment as well

if GROUP_CONCAT() function is not available in a database system, the exact same line can be run but replacing GROUP_CONCAT with COUNT to instead get the number of files attached

In [8]:
%%time
# faster subquery with joins - apparently better than just using WHERE
q = '''
SELECT e.id, e.subject, e.internal_date, e.from_address, e.'to', e.bcc, e.cc, e.reply_to, e.html_body, e.plain_text_body,
       GROUP_CONCAT(CASE WHEN a.original_filename IS NOT NULL THEN a.original_filename ELSE NULL END) AS attachments,
       c.id AS category_id, c.slug AS category_slug
FROM emails_email e
JOIN emails_emailattachment a ON e.id = a.email_id
JOIN emails_emailcategory ec ON e.id = ec.email_id
JOIN emails_category c ON c.id = ec.category_id
WHERE ec.category_id IS NOT NULL
GROUP BY e.id, e.subject, e.internal_date, e.from_address, e.'to', e.bcc, e.cc, e.reply_to, c.id, c.slug
'''

# Execute the query and store the results in a DataFrame
df = pd.read_sql_query(q,con)
labelled_emails = df
labelled_emails

,id,subject,internal_date,from_address,to,bcc,cc,reply_to,html_body,plain_text_body,attachments,category_id,category_slug
0,167529,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,2021-11-17 16:08:00+00,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html>\n<head>\n<meta http-equiv=""Content-Type...",None,"19427_2422423_00112742.pdf,19427_2422423_00112...",100,invoices
1,167544,"Overdue Account £59,632.58",2022-07-05 15:37:48+00,sefika@albion.co.uk,{Matt.Hales@evo-group.co.uk},{},{Ryan.Leigh@evo-group.co.uk},{},"<html>\n<head>\n<meta http-equiv=""Content-Type...",None,"Followups(6).pdf,Followups(6).pdf,Followups(6)...",1,statement
2,170741,FW: APRIL ACCOUNT,2021-07-28 10:10:32+00,Lisa.Dyson@evo-group.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None,"CR1018258.pdf,CR1018258.pdf,CR1018259.pdf,CR10...",100,invoices
3,170766,None,2021-07-28 11:03:11+00,clovell@ups.com,{purchase.ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",Attention: This email originated outside of ou...,"scr019205.pdf,scr019205.pdf,scr019205.pdf,scr0...",100,invoices
4,172273,ban 1148 Overdue Account,2021-08-02 09:08:05+00,PennyRichardson@morleys.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",Attention: This email originated outside of ou...,"ban-1148.pdf,ban-1148.pdf,image001.png,image00...",101,reminders
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14328,390994,INM323435 Invoice from Dempson Ltd,2022-09-07 17:30:12+00,info@dempson.co.uk,{purchase.ledger@evo-group.co.uk},{},{},{info@dempson.co.uk},"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",None,"Attachment.pdf,Attachment.pdf",100,invoices
14329,391263,STP Invoice 0090343971,2022-09-07 08:21:03+00,Lara.Poehlmann@stp.de,{purchase.ledger@evo-group.co.uk},{},{},{},None,Attention: This email originated outside of ou...,"STP Invoice 0090343971.pdf,STP Invoice 0090343...",100,invoices
14330,393461,W-9,2022-09-12 16:16:35+00,ksmith@3rivers.net,{APCustomerService@scripps.com},{},{},{},"<html><head>\n<meta http-equiv=""Content-Type"" ...",None,"W-9.pdf,W-9.pdf",47,other-queries
14331,406564,RE: Statement of Account for your Amazon Adver...,2022-10-11 19:08:39+00,srudrama@amazon.com,{APCustomerService@scripps.com},{},{advertising-receivables@amazon.com},{},"<html><head>\n<meta http-equiv=""Content-Type"" ...",None,"image001.png,image001.png,image002.jpg,image00...",101,reminders


In [10]:
# cleaning up memory throughout
del df

NameError: name 'df' is not defined

In [ ]:
# this didn't work
# q = '''
# SELECT e.id, e.subject, e.internal_date, e.'to', 
#         IFNULL((SELECT GROUP_CONCAT(DISTINCT original_filename SEPARATOR '; ')
#                FROM emails_emailattachment a
#                WHERE a.email_id = e.id AND original_filename IS NOT NULL), '') AS has_attachment,
#        c.id AS category_id, c.slug AS category_slug 
# FROM emails_email e 
# JOIN emails_emailcategory ec ON e.id = ec.email_id 
# JOIN emails_category c ON c.id = ec.category_id 
# LEFT JOIN emails_emailattachment a ON e.id = a.email_id 
# WHERE ec.category_id IS NOT NULL 
# GROUP BY e.id, e.subject, e.internal_date, c.id, c.slug;
# '''

# # Execute the query and store the results in a DataFrame
# labelled_emails = pd.read_sql_query(q,con)
# labelled_emails

### Using this subquery, I get out 14,333 labelled emails - so a little less than before, because I think I'm removing the duplicates here now

In [11]:
print(len(labelled_emails))
labelled_emails.head(5)

14333


,id,subject,internal_date,from_address,to,bcc,cc,reply_to,html_body,plain_text_body,attachments,category_id,category_slug
0,167529,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,2021-11-17 16:08:00+00,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html>\n<head>\n<meta http-equiv=""Content-Type...",None,"19427_2422423_00112742.pdf,19427_2422423_00112...",100,invoices
1,167544,"Overdue Account £59,632.58",2022-07-05 15:37:48+00,sefika@albion.co.uk,{Matt.Hales@evo-group.co.uk},{},{Ryan.Leigh@evo-group.co.uk},{},"<html>\n<head>\n<meta http-equiv=""Content-Type...",None,"Followups(6).pdf,Followups(6).pdf,Followups(6)...",1,statement
2,170741,FW: APRIL ACCOUNT,2021-07-28 10:10:32+00,Lisa.Dyson@evo-group.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None,"CR1018258.pdf,CR1018258.pdf,CR1018259.pdf,CR10...",100,invoices
3,170766,None,2021-07-28 11:03:11+00,clovell@ups.com,{purchase.ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",Attention: This email originated outside of ou...,"scr019205.pdf,scr019205.pdf,scr019205.pdf,scr0...",100,invoices
4,172273,ban 1148 Overdue Account,2021-08-02 09:08:05+00,PennyRichardson@morleys.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",Attention: This email originated outside of ou...,"ban-1148.pdf,ban-1148.pdf,image001.png,image00...",101,reminders


## The below works fine, except doesn't include the email_attachment info

And it takes a while to run, might also want to limit the number of fields imported from emails_email 

In [ ]:
# SQL code to extract all bank-detail-change emails
# extract id and slug from emails_category
# link to id == category_id in emails_emailcategory, extract email_id, category_id
# link email_id from emails_emailcategory to id in emails_email

# q = '''
#     SELECT emails_category.id, emails_category.slug,
#         emails_emailcategory.email_id, emails_emailcategory.category_id,
#         emails_email.*
#     FROM emails_category, emails_emailcategory, emails_email
#     WHERE (emails_category.id = emails_emailcategory.category_id) AND 
#     (emails_emailcategory.email_id = emails_email.id) AND 
#     emails_emailcategory.category_id IS NOT NULL
# '''

# labelled_emails = pd.read_sql_query(q,con)
# labelled_emails

In [12]:
print(labelled_emails.category_slug.unique())

['invoices' 'statement' 'reminders' 'bank-detail-change' 'payments'
 'other-queries' 'invoices-subject-line-pdf' 'new-supplier'
 'invoices-content' 'invoice-no-tags' 'reminder-pdf' 'no-action'
 'order-dispatch-and-tracking' 'order-tracking' 'credit-note'
 'order-acknowledgement' 'backorder' 'reminder-content'
 'invoices-subject-line-pdf-other' 'direct-debit-advice-reminder' 'demo'
 'invoices-subject-line-excel' 'shipping-manifest'
 'invoices-subject-line-word']


So it seems that there are 31,410 emails that are classified, as one of the categories above

In [13]:
labelled_emails['category_slug'].value_counts()

invoices                           5673
invoices-subject-line-pdf          4356
statement                          1417
demo                                684
reminders                           383
invoice-no-tags                     345
order-tracking                      240
no-action                           152
payments                            140
other-queries                       127
reminder-pdf                        120
credit-note                         116
order-acknowledgement               103
invoices-content                     99
new-supplier                         90
order-dispatch-and-tracking          86
invoices-subject-line-pdf-other      72
reminder-content                     61
bank-detail-change                   25
direct-debit-advice-reminder         19
backorder                            16
shipping-manifest                     4
invoices-subject-line-excel           3
invoices-subject-line-word            2
Name: category_slug, dtype: int64

# Check for identical columns we can drop

Not needed anymore with new form of SQL subquery

In [ ]:
# # should be 19 columns left
# print(len(labelled_emails.columns))
# labelled_emails.columns

In [ ]:
# # email_id and email_id
# print(labelled_emails.columns[2])
# print(labelled_emails.columns[4])
# labelled_emails.iloc[:,2].equals(labelled_emails.iloc[:,4])

In [ ]:
# labelled_emails.columns.values[4] = "duplicate_email_id"

In [ ]:
## this is only duplicated when including email_attachment table
# email_id and id
# print(labelled_emails.columns[4])
# print(labelled_emails.columns[7])
# labelled_emails.iloc[:,4].equals(labelled_emails.iloc[:,7])

In [ ]:
# labelled_emails.columns.values[7] = "duplicate_email_id2"

In [ ]:
# # id and category_id
# print(labelled_emails.columns[0])
# print(labelled_emails.columns[3])
# labelled_emails.iloc[:,0].equals(labelled_emails.iloc[:,3])

In [ ]:
# labelled_emails.columns.values[0] = "duplicate_category_id"

In [ ]:
# # should be 19 columns left
# print(len(labelled_emails.columns))
# labelled_emails.columns

In [ ]:
# # labelled_emails = labelled_emails.drop(columns = ["duplicate_email_id","duplicate_email_id2","duplicate_category_id"])
# labelled_emails = labelled_emails.drop(columns = ["duplicate_email_id","duplicate_category_id"])

In [ ]:
# # should be 19 columns left
# print(len(labelled_emails.columns))
# labelled_emails.columns

## Cleaning the dataframe a bit more 

- Recoding some columns

- Check for duplicates: There should be 0 now

In [ ]:
# labelled_emails['email_id'].value_counts()

In [14]:
print(sum(labelled_emails.duplicated()))

0


In [ ]:
# # there seem to be 15416 duplicates
# labelled_emails.drop_duplicates(keep = 'first', inplace = True)
# labelled_emails

In [ ]:
# print(len(labelled_emails))

## Clean the dataset some more

In [ ]:
#labelled_emails = df

In [17]:
# try and import my own functions:
import sys
import ipynb
sys.path.append('/project/Xelix_Project/utils')

#from ipynb.fs.full.Regex_html_Functions import remove_duplicates

from ipynb.fs.full.Regex_html_Functions import remove_duplicates, clean_attachment_types

In [18]:
import re

In [ ]:
# function to remove duplicates in string
# def remove_duplicates(s):
#     pattern = ['.png','.jpg']
#     if s:
#         my_list = sorted(set(s.split(',')))
#         filtered_items = [item for item in my_list if pattern not in item]
#         return ','.join(filtered_items)
#     else:
#         return ''

In [ ]:
# # function to remove duplicates in string
# def remove_duplicates(s):
#     '''Function to remove duplicate strings in a list'''
#     if s:
#         return ','.join(sorted(set(s.split(','))))
#     else:
#         return ''

In [19]:
labelled_emails.head(1)

,id,subject,internal_date,from_address,to,bcc,cc,reply_to,html_body,plain_text_body,attachments,category_id,category_slug
0,167529,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,2021-11-17 16:08:00+00,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html>\n<head>\n<meta http-equiv=""Content-Type...",None,"19427_2422423_00112742.pdf,19427_2422423_00112...",100,invoices


In [20]:
labelled_emails['attachments'] = labelled_emails['attachments'].apply(remove_duplicates)

In [21]:
labelled_emails['attachments'].unique()

array(['19427_2422423_00112742.pdf,image165943.png,image318233.png,image463652.png',
       'Followups(6).pdf',
       'CR1018258.pdf,CR1018259.pdf,CR1018260.pdf,CR1018320.pdf,CR1018324.pdf,CR1018334.pdf,CR1018337.pdf,CR1018338.pdf,CR1018339.pdf,CR1018342.pdf,CR1018418.pdf,CR1018419.pdf,EDI002270.pdf,EDI002271.pdf,EDI002272.pdf,EDI002278.pdf,EDI002279.pdf,EDI002280.pdf,EDI002281.pdf,EDI002282.pdf,SI1477307.pdf,image001.png,image002.jpg,image003.jpg',
       ...,
       '04336836571352041FC1A2DDA7CC73D3D23C4C57.png,1B7B56FDD4310A10C55C32978C1B8D18BCE1A2E9.css,20220901_ UK_4008964.PDF,57128B7074B001AC241925D8B5CEDED9BDE12BB8.png',
       'STP Invoice 0090343971.pdf', 'W-9.pdf'], dtype=object)

In [22]:
# # function to remove duplicates in string
# def clean_attachment_types(s):
#     '''Function to remove strings with certain suffixes from a list'''
#     suffixes = ['.png','.jpg','.jpeg','.css']
#     if not isinstance(s, list):
#         s = [s]
#     result = []
#     for item in s:
#         if not any(suffix in item for suffix in suffixes):
#             result.append(item)
#     return result if len(result) > 1 else result[0] if result else None

In [23]:
labelled_emails['attachments'] = labelled_emails['attachments'].apply(clean_attachment_types)

In [24]:
labelled_emails['attachments']

0                              None
1                  Followups(6).pdf
2                              None
3                     scr019205.pdf
4                              None
                    ...            
14328                Attachment.pdf
14329    STP Invoice 0090343971.pdf
14330                       W-9.pdf
14331                          None
14332                          None
Name: attachments, Length: 14333, dtype: object

In [25]:
# recode column
labelled_emails['has_attachment'] = [True if val is not None else False for val in labelled_emails['attachments']]

In [26]:
labelled_emails.head(5)

,id,subject,internal_date,from_address,to,bcc,cc,reply_to,html_body,plain_text_body,attachments,category_id,category_slug,has_attachment
0,167529,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,2021-11-17 16:08:00+00,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html>\n<head>\n<meta http-equiv=""Content-Type...",None,None,100,invoices,False
1,167544,"Overdue Account £59,632.58",2022-07-05 15:37:48+00,sefika@albion.co.uk,{Matt.Hales@evo-group.co.uk},{},{Ryan.Leigh@evo-group.co.uk},{},"<html>\n<head>\n<meta http-equiv=""Content-Type...",None,Followups(6).pdf,1,statement,True
2,170741,FW: APRIL ACCOUNT,2021-07-28 10:10:32+00,Lisa.Dyson@evo-group.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",None,None,100,invoices,False
3,170766,None,2021-07-28 11:03:11+00,clovell@ups.com,{purchase.ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",Attention: This email originated outside of ou...,scr019205.pdf,100,invoices,True
4,172273,ban 1148 Overdue Account,2021-08-02 09:08:05+00,PennyRichardson@morleys.co.uk,{Purchase.Ledger@evo-group.co.uk},{},{},{},"<html xmlns:v=""urn:schemas-microsoft-com:vml"" ...",Attention: This email originated outside of ou...,None,101,reminders,False


# Clean the html body text

A function to clean the text

In [27]:
#w_html = df.html_body.notnull() 
#w_plain = df.plain_text_body.notnull()


from ipynb.fs.full.Regex_html_Functions import text_from_html

In [28]:
# from bs4 import BeautifulSoup, Comment

# def text_from_html(body) -> str:
#     if pd.isnull(body):
#         return ""

#     def tag_visible(element):
#         if element.parent.name in ["style", "script", "head", "title", "meta", "[document]"]:
#             return False
#         if isinstance(element, Comment):
#             return False
#         return True

#     soup = BeautifulSoup(body, "html.parser")
#     texts = soup.findAll(string=True)
#     visible_texts = filter(tag_visible, texts)
#     return " ".join(t.strip() for t in visible_texts)

# # import html2text
# # html2text.html2text(text)

# assess the data a little bit

In [29]:
# I seem to find 31,410 labelled emails
print(f'Number of labelled emails: {len(labelled_emails)}')

Number of labelled emails: 14333


In [30]:
bankchange_emails = labelled_emails[labelled_emails.category_slug.str.contains("bank-detail-change") == True]
print(f'Number of bank-detail-change requesting emails: {len(bankchange_emails)}')

Number of bank-detail-change requesting emails: 25


Just checking again that there seem to be 54 bank detail change emails in the dataset

- UPDATE: with the updated SQL subquery it's now 25

In [31]:
labelled_emails.columns

Index(['id', 'subject', 'internal_date', 'from_address', 'to', 'bcc', 'cc',
       'reply_to', 'html_body', 'plain_text_body', 'attachments',
       'category_id', 'category_slug', 'has_attachment'],
      dtype='object')

## Run below cell by cell, just to make sure no errors pop up

Got some errors from text_from_html function before (but there shouldn't be any)

(Nevermind this was a silly error - forgot to rename df)

In [ ]:
%%time
# clean just the bank_deets emails' body of text - so we can look at them
labelled_emails["html_body_as_text"] = labelled_emails["html_body"].map(text_from_html)
labelled_emails["body"] = labelled_emails["plain_text_body"]
labelled_emails.loc[labelled_emails["body"].isnull(), "body"] = labelled_emails.loc[labelled_emails["body"].isnull(), "html_body_as_text"]

The above lines (mainly the text_from_html function) take 1 min 2 seconds to run. 

In [ ]:
#labelled_emails.loc[labelled_emails["body"].isnull(), "body"] = labelled_emails.loc[labelled_emails["body"].isnull(), "html_body_as_text"]

In [ ]:
labelled_emails

In [ ]:
# check all names in the data again
labelled_emails.columns

### Use pickle to export some objects

Use this because it's fast and preserves data

to turn into pickle:
- original_df.to_pickle("./dummy.pkl")

to import and read again:

- unpickled_df = pd.read_pickle("./dummy.pkl")

In [ ]:
import pickle

In [ ]:
bankchange_emails = labelled_emails[labelled_emails.category_slug.str.contains("bank-detail-change") == True]
len(bankchange_emails)

In [ ]:
labelled_emails.to_pickle("categorised_emails_and_threads.pkl")
bankchange_emails.to_pickle("bank_change_emails_and_threads.pkl")

In [32]:
labelled_emails = pd.read_pickle("categorised_emails_and_threads.pkl")
bankchange_emails = labelled_emails[labelled_emails.category_slug.str.contains("bank-detail-change") == True]
len(bankchange_emails)

25

In [ ]:
# for row in df[df.plain_text_body.str.contains("bank details") == True].head(10).itertuples():
# for row in df.head(100).itertuples():
#     print("#" * 70, "\n\n")
#     print(row.subject)
#     print("")
#     print(row.body)
#     print("\n")

In [ ]:
# to read the files again:
#labelled_emails = pd.read_pickle("categorised_emails.pkl")
#bankchange_emails = pd.read_pickle("bank_change_emails.pkl")

Below runs this on the larger datasets if extracted

In [ ]:
# # this step takes pretty long to run - especially if the data is very long
# df["html_body_as_text"] = df["html_body"].map(text_from_html)
# df["body"] = df["plain_text_body"]
# df.loc[df["body"].isnull(), "body"] = df.loc[df["body"].isnull(), "html_body_as_text"]

In [ ]:
# # for row in df[df.plain_text_body.str.contains("bank details") == True].head(10).itertuples():
# for row in df.head(100).itertuples():
#     print("#" * 70, "\n\n")
#     print(row.subject)
#     print("")
#     print(row.body)
#     print("\n")

# Look at the data
How many emails? How many are classified right now?

In [33]:
labelled_emails.columns

Index(['id', 'subject', 'internal_date', 'from_address', 'to', 'bcc', 'cc',
       'reply_to', 'html_body', 'plain_text_body', 'attachments',
       'category_id', 'category_slug', 'has_attachment', 'html_body_as_text',
       'body'],
      dtype='object')

In [34]:
print(len(labelled_emails))

14333


1 - how many are labelled as bank detail changes? What do they look like?

In [35]:
bankchange_emails.columns

Index(['id', 'subject', 'internal_date', 'from_address', 'to', 'bcc', 'cc',
       'reply_to', 'html_body', 'plain_text_body', 'attachments',
       'category_id', 'category_slug', 'has_attachment', 'html_body_as_text',
       'body'],
      dtype='object')

In [36]:
bankchange_emails["attachments"].unique()

array([None, 'Change of Company Bank Details.pdf',
       'Innovid Bank Name Change Notification Letter - Signed.pdf',
       'ACH Keybank Letterhead (6).pdf', 'EFT_FORM_US.pdf',
       '12.15.20.pdf', 'Scanned Document.pdf'], dtype=object)

In [ ]:
#bankchange_emails.iloc[0].body

In [ ]:
#split_string = "From: "
#split_emails = bankchange_emails.iloc[0].body.split(split_string)

In [ ]:
#split_emails

In [ ]:
#split_emails[1].split("Sent: ")[0]

## Save separate emails as rows in a new dataframe 

For now let's just go through each email and save it as a row in a new dataframe. First try this with the bank detail change emails and then expand to doing this for all labelled emails, then all the other ones?

## Some Regex functions

The one below cleans hyperlinks, curly brackets etc from the body of the email text

In [ ]:
from ipynb.fs.full.Regex_html_Functions import (clean_text_re_links_brackets, find_sort_code_in_email,
                                               find_bank_vocabulary_mention_in_email, find_bank_mention_in_email,
                                               find_update_words_in_email)

In [ ]:
# # to do with Regex. Clean first:

# # remove phone numbers
# # find if it has a sort account
# # find if the body of the text contains "change", "new"
# # potentially save if hyperlink was present in email?

# import re 
# import string

# def clean_text_re_links_brackets(text):
#     '''Clean text in following ways:
#     - remove text in square brackets [] and within a html tag <>
#     - remove all https links
#     - find if sort code included in email
#     ? find if the body of the email contains "bank","change","new","sort code"
#     '''
    
#     text = re.sub('\[.*?\]', '', text) # clean text from within square brackets 
#     text = re.sub('\<.*?\>', '', text) # clean text from inside hyperlinks
#     text = re.sub('\{.*?\}', '', text) # clean text from inside curly brackets
#     text = re.sub('\{\n}', '', text)
    
#     return text


The function below logs whether the email contains a sort code, and whether it mentions a sort code, and extracts it if so

In [ ]:
# # log if email contains sort code and store it if so
# def find_sort_code_in_email(text):
#     '''Find whether email contains a sort code with simple RegEx function:
#     - extract sort code numbers (only if it matches format of dd-dd-dd)
#     '''
#     # the \b stands for word boundary. this r
#     #srt_account_string = re.compile(r'\b\d{2}-?\d{2}-?\d{2}\b') # this will accept both with and without dashes
#     srt_account_string = re.compile(r'\b\d{2}-\d{2}-\d{2}\b') # this will hardcode in the dashes
#     match = srt_account_string.search(text)
#     if match:
#         return True, match.group()
#     else:
#         return False, None
    
#     # if only want to return a single True/False output:
# #     return bool(srt_account_string.search(text))

In [ ]:
# # write function to find if the text mentions bank account or bank details
# def find_bank_vocabulary_mention_in_email(text):
#     '''Find whether email contains certain words which might relate to bank-detail-change emails with simple RegEx function:
#     - mentions sort-code, IBAN or swift
#     '''
#     # the \b stands for word boundary. this r
#     srt_account_string = re.compile(r'sort code|IBAN|swift', re.IGNORECASE)
#     match = srt_account_string.search(text)
#     if match:
#         return True #, match.group()
#     else:
#         return False

In [ ]:
# # write function to find if the text mentions bank account or bank details
# def find_bank_mention_in_email(text):
#     '''Find whether email contains certain words which might relate to bank-detail-change emails with simple RegEx function:
#     - mentions bank account or bank details
#     '''
#     # the \b stands for word boundary. this r
#     srt_account_string = re.compile(r'bank account|bank details', re.IGNORECASE)
#     match = srt_account_string.search(text)
#     if match:
#         return True #, match.group()
#     else:
#         return False

In [ ]:
# # write function to find if the text mentions change or new or updated
# def find_update_words_in_email(text):
#     '''Find whether email contains certain words which might relate to bank-detail-change emails with simple RegEx function:
#     - mentions change, new or updated
#     '''
#     # the \b stands for word boundary. this r
#     srt_account_string = re.compile(r'change|new|updated', re.IGNORECASE)
#     match = srt_account_string.search(text)
#     if match:
#         return True #, match.group()
#     else:
#         return False

In [ ]:
#bankchange_emails['to'][12]

In [ ]:
#bankchange_emails['to'][12].count(',')+1

In [ ]:
#text = bankchange_emails['body'][4839]
#find_update_words_in_email(text)

In [ ]:
#find_bank_mention_in_email(text)

In [ ]:
# assign new dataframe names
# columns = ['id', 'subject', 'internal_date', 'from_address', 'from_name','to', 'bcc', 'cc',
#        'reply_to', 'attachments','category_id', 'category_slug', 'has_attachment','body',
#           'is_partof_thread','num_in_thread','thread_id']


In [ ]:
# this doesnt work
# from datetime import datetime

# def convert_date(date):
#     '''Convert a string to a datetime object'''
#     datetime.fromisoformat
#     date_time_str = date.strftime("%Y-%m-%d %H:%M:%S")
#     return date_time_str

# Just display emails (no splitting)

In [ ]:
# select 26 or 27
iter_count = 1
for index, row in labelled_emails.iterrows():
    iter_count += 1
    if iter_count <30:
        print("#" * 70, "\n\n")
        print(f'email num: {index}')

        print(f'subject line:\t{row.subject}')
        print(f'received from:\t{row.from_address}')
        print(f'sent to:\t{row.to}')
        print(f'date sent:\t{row.internal_date}')
        print(f'slug:\t\t{row.category_slug}')
        print(row.body)

In [ ]:
iter_count = 1
for row in labelled_emails.itertuples():
    print("#" * 70, "\n\n")
    print(f'email num: {iter_count}')
    iter_count += 1
    print(f'subject line:\t{row.subject}')
    print(f'received from:\t{row.from_address}')
    print(f'sent to:\t{row.to}')
    print(f'date sent:\t{row.internal_date}')
    print(f'slug:\t\t{row.category_slug}')
    print(row.body)

# Split the emails, display them and store them

In [ ]:
# grow lists iteratively, then create dataframe at the end
email_id = []
subject = []
internal_date = []
from_address = []
from_name = []
sent_to = []
sent_to_how_many = []
bcc = []
cc = []
reply_to_address = []
attachments = []
category_id = []
category_slug = []
has_attachment = []
email_text_body = []

# new columns for splitting the emails
is_part_of_thread = []
num_in_thread = []
thread_id = []

# sort code related
has_sort_code = []
sort_code = []

# language related
mentions_bank = []
mentions_change = []
mentions_account_specifics = []

############################
# splitting emails by this: 
# Disclaimer: need to check this is the same across languages etc
split_string = "From: "
original_message_string = "-----Original Message-----"

iter_count = 1

# change name of df here
for line in labelled_emails.itertuples():
    
    print("#" * 70, "\n\n")
    print(f'email {iter_count} in dataframe')
    iter_count += 1
    line.body = re.sub(original_message_string,"",line.body)
    
    ################### if single email, we just append as we would
    if split_string not in line.body:
        
        # save thread details
        is_part_of_thread.append(0)
        num_in_thread.append(None)
        thread_id.append(None)
        
        # save data
        email_id.append(line.id)
        attachments.append(line.attachments)
        category_id.append(line.category_id)
        category_slug.append(line.category_slug)
        has_attachment.append(line.has_attachment)
        
        # email specific
        subject.append(line.subject)
        internal_date.append(line.internal_date)
        from_address.append(line.from_address)
        from_name.append(line.from_address) # just save sender email instead here
        sent_to.append(line.to)
        sent_to_how_many.append(line.to.count(',')+1)
        bcc.append(line.bcc)
        cc.append(line.cc)
        reply_to_address.append(line.reply_to)
        
#         attachments.append(line.attachments)
#         category_id.append(line.category_id)
#         category_slug.append(line.category_slug)
#         has_attachment.append(line.has_attachment)
        
        sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
        has_sort_code.append(sort_code_bool)
        sort_code.append(sort_code_num)

        # language related
        mentions_bank.append(find_bank_mention_in_email(line.body))
        mentions_change.append(find_update_words_in_email(line.body))
        mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

        print(f'subject line:\t{line.subject}')
        print(f'received from:\t{line.from_address}')
        print(f'sent to:\t{line.to}')
        #print(f'emails sent to: {line.to.count(',')+1} people')
        print(f'date sent:\t{line.internal_date}')
        print(f'slug:\t\t{line.category_slug}')
        print("")
        print(clean_text_re_links_brackets(line.body))
        print("\n")
        
        # clean email body
        email_text_body.append(clean_text_re_links_brackets(line.body))
        
    ################### thread of emails
    else:
        split_emails = line.body.split("From: ")
        
        for i, row in enumerate(split_emails):
            
             # save thread details
            is_part_of_thread.append(1)
            num_in_thread.append(int(len(split_emails)-(i+1)))
            thread_id.append(str(line.id)+'_t')
            
            # save regardless of thread or not
            email_id.append(line.id)
            attachments.append(line.attachments)
            category_id.append(line.category_id)
            category_slug.append(line.category_slug)
            has_attachment.append(line.has_attachment)
            
            # email specific
#             subject.append(line.subject)
#             internal_date.append(line.internal_date)
#             from_address.append(line.from_address)
#             from_name.append(line.from_address) # just save sender email instead here
#             sent_to.append(line.to)
#             bcc.append(line.bcc)
#             cc.append(line.cc)
            bcc.append(None)
            cc.append(None)
            reply_to_address.append(line.reply_to) # just keep this consistent?
            
            # for every split email which wasn't the first
            if i > 0:
                before_sent = row.split("Sent: ")[0]
                print(before_sent,'\n')
                #print(f'From: {this_person}, {this_email}')
                this_email = re.findall(r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])', before_sent)
                print(this_email)
                
                row = row.split("Sent: ")[1]
                if not(this_email):
                    this_person = before_sent
                    from_address.append(this_person) # save output
                else:
                    from_address.append(this_email) # save output
                    this_person = re.sub(r'<(.*?)>',"",before_sent)  
                    
                from_name.append(this_person) # save output
                start_index = row.find('Sent: ') + len('Sent: ')
                end_index = row.find('To: ')
                date_text = row[start_index:end_index]
                internal_date.append(date_text) # save output
                
                row = re.sub(date_text,"",row)
                print(f'Date: {date_text}')

                start_index = row.find('To: ') + len('To: ')
                end_index = row.find('Subject: ')
                to_text = row[start_index:end_index]
                to_emails = re.findall(r'<(.*?)>', to_text)
                sent_to.append(to_emails) # save output
                row = re.sub(to_text,"",row)
                print(f'To: {to_emails}')
                print(f'emails sent to: {len(to_emails)} people')
                sent_to_how_many.append(len(to_emails)) # save output
                start_index = row.find('Subject: ') + len('Subject: ')
                end_index = row.find('      ')
                this_subject = row[start_index:end_index]
                print(f'subject: {this_subject}')
                subject.append(this_subject) # save output
                
                row = re.sub('Sent: ',"",row)
                row = re.sub('To: ',"",row)
                row = re.sub('Subject: ',"",row)
                row = re.sub(this_subject,"",row)
                

                
                # conditionals
                sort_code_bool, sort_code_num = find_sort_code_in_email(row)
                has_sort_code.append(sort_code_bool)
                sort_code.append(sort_code_num)

                # language related
                mentions_bank.append(find_bank_mention_in_email(row))
                mentions_change.append(find_update_words_in_email(row))
                mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

                print('\n')
                print(clean_text_re_links_brackets(row))
                print('\n')
                
                # clean email body
                email_text_body.append(clean_text_re_links_brackets(row))
                
            elif i == 0:
                print(f'subject line:\t{line.subject}')
                print(f'received from:\t{line.from_address}')
                print(f'sent to:\t{line.to}')
                #print(f'emails sent to: {line.to.count(',')+1} people')
                print(f'date sent:\t{line.internal_date}')
                print(f'slug:\t\t{line.category_slug}')
                
                
                subject.append(line.subject)
                internal_date.append(line.internal_date)
                from_address.append(line.from_address)
                from_name.append(line.from_address) # just save sender email instead here
                sent_to.append(line.to)
                sent_to_how_many.append(line.to.count(',')+1)
                
                sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
                has_sort_code.append(sort_code_bool)
                sort_code.append(sort_code_num)

                # language related
                mentions_bank.append(find_bank_mention_in_email(line.body))
                mentions_change.append(find_update_words_in_email(line.body))
                mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
                
                # clean email body
                email_text_body.append(clean_text_re_links_brackets(row))

                print("")
                print(clean_text_re_links_brackets(row))
                print('\n')

# Don't display, just iterate and save features

In [ ]:
# grow lists iteratively, then create dataframe at the end
email_id = []
subject = []
internal_date = []
from_address = []
from_name = []
sent_to = []
sent_to_how_many = []
bcc = []
cc = []
reply_to_address = []
attachments = []
category_id = []
category_slug = []
has_attachment = []
email_text_body = []

# new columns for splitting the emails
is_part_of_thread = []
num_in_thread = []
thread_id = []

# sort code related
has_sort_code = []
sort_code = []

# language related
mentions_bank = []
mentions_change = []
mentions_account_specifics = []

############################
# splitting emails by this: 
# Disclaimer: need to check this is the same across languages etc
split_string = "From: "

for line in bankchange_emails.itertuples():
        
    ################### if single email, we just append as we would
    if split_string not in line.body:
        
        # save thread details
        is_part_of_thread.append(0)
        num_in_thread.append(None)
        thread_id.append(None)
        
        # save data
        email_id.append(line.id)
        attachments.append(line.attachments)
        category_id.append(line.category_id)
        category_slug.append(line.category_slug)
        has_attachment.append(line.has_attachment)
        
        # email specific
        subject.append(line.subject)
        internal_date.append(line.internal_date)
        from_address.append(line.from_address)
        from_name.append(line.from_address) # just save sender email instead here
        sent_to.append(line.to)
        sent_to_how_many.append(line.to.count(',')+1)
        bcc.append(line.bcc)
        cc.append(line.cc)
        reply_to_address.append(line.reply_to)
        
#         attachments.append(line.attachments)
#         category_id.append(line.category_id)
#         category_slug.append(line.category_slug)
#         has_attachment.append(line.has_attachment)
        
        sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
        has_sort_code.append(sort_code_bool)
        sort_code.append(sort_code_num)

        # language related
        mentions_bank.append(find_bank_mention_in_email(line.body))
        mentions_change.append(find_update_words_in_email(line.body))
        mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
        
        # clean email body
        email_text_body.append(clean_text_re_links_brackets(line.body))
        
    ################### thread of emails
    else:
        split_emails = line.body.split("From: ")
        
        for i, row in enumerate(split_emails):
            
             # save thread details
            is_part_of_thread.append(1)
            num_in_thread.append(int(len(split_emails)-(i+1)))
            thread_id.append(str(line.id)+'_t')
            
            # save regardless of thread or not
            email_id.append(line.id)
            attachments.append(line.attachments)
            category_id.append(line.category_id)
            category_slug.append(line.category_slug)
            has_attachment.append(line.has_attachment)
            
            # email specific
#             subject.append(line.subject)
#             internal_date.append(line.internal_date)
#             from_address.append(line.from_address)
#             from_name.append(line.from_address) # just save sender email instead here
#             sent_to.append(line.to)
#             bcc.append(line.bcc)
#             cc.append(line.cc)
            bcc.append(None)
            cc.append(None)
            reply_to_address.append(line.reply_to) # just keep this consistent?
            
            # for every split email which wasn't the first
            if i > 0:
                before_sent = row.split("Sent: ")[0]
                #print(before_sent,'\n')
                #print(f'From: {this_person}, {this_email}')
                this_email = re.findall(r'<(.*?)>', before_sent)
                row = re.sub(before_sent,"",row)
                if not(this_email):
                    this_person = before_sent
                    from_address.append(this_person) # save output
                else:
                    from_address.append(this_email) # save output
                    this_person = re.sub(r'<(.*?)>',"",before_sent)  
                    
                from_name.append(this_person) # save output
                start_index = row.find('Sent: ') + len('Sent: ')
                end_index = row.find('To: ')
                date_text = row[start_index:end_index]
                internal_date.append(date_text) # save output
                
                row = re.sub(date_text,"",row)

                start_index = row.find('To: ') + len('To: ')
                end_index = row.find('Subject: ')
                to_text = row[start_index:end_index]
                to_emails = re.findall(r'<(.*?)>', to_text)
                sent_to.append(to_emails) # save output
                row = re.sub(to_text,"",row)
                sent_to_how_many.append(len(to_emails)) # save output
                start_index = row.find('Subject: ') + len('Subject: ')
                end_index = row.find('      ')
                this_subject = row[start_index:end_index]
                subject.append(this_subject) # save output
                
                row = re.sub('Sent: ',"",row)
                row = re.sub('To: ',"",row)
                row = re.sub('Subject: ',"",row)
                row = re.sub(this_subject,"",row)
                

                
                # conditionals
                sort_code_bool, sort_code_num = find_sort_code_in_email(row)
                has_sort_code.append(sort_code_bool)
                sort_code.append(sort_code_num)

                # language related
                mentions_bank.append(find_bank_mention_in_email(row))
                mentions_change.append(find_update_words_in_email(row))
                mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))
                
                # clean email body
                email_text_body.append(clean_text_re_links_brackets(row))
                
            # first email in thread - need to change something here
            elif i == 0:
                
                subject.append(line.subject)
                internal_date.append(line.internal_date)
                from_address.append(line.from_address)
                from_name.append(line.from_address) # just save sender email instead here
                sent_to.append(line.to)
                sent_to_how_many.append(line.to.count(',')+1)
                
                sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
                has_sort_code.append(sort_code_bool)
                sort_code.append(sort_code_num)

                # language related
                mentions_bank.append(find_bank_mention_in_email(line.body))
                mentions_change.append(find_update_words_in_email(line.body))
                mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
                
                # clean email body
                email_text_body.append(clean_text_re_links_brackets(row))


In [ ]:
dict_data = {'email_id':email_id,'is_part_of_thread':is_part_of_thread,'num_in_thread':num_in_thread,'thread_id':thread_id,
             'internal_date':internal_date,'category_id':category_id,'category_slug':category_slug,'subject':subject,
            'has_attachment':has_attachment,'has_sort_code':has_sort_code,'sort_code':sort_code,
             'mentions_bank':mentions_bank,'mentions_change':mentions_change,'mentions_account_specifics':mentions_account_specifics,
             'from_address':from_address,'from_name':from_name,'sent_to':sent_to,'sent_to_how_many':sent_to_how_many,
             'bcc':bcc,'cc':cc,'reply_to_address':reply_to_address,'attachments':attachments,'email_text_body':email_text_body
            }

print(len(dict_data))

In [ ]:
new_df = pd.DataFrame.from_dict(dict_data)

In [ ]:
new_df

# Store new features as pickle

In [ ]:
new_df.to_pickle("bank_change_emails_split_newfeatures.pkl")
#bankchange_emails.to_pickle("bank_change_emails_and_threads.pkl")

In [38]:
new_df = pd.read_pickle("bank_change_emails_split_newfeatures.pkl")

In [41]:
pd.set_option('display.max_columns', 500)
new_df.head(25)

,email_id,is_part_of_thread,num_in_thread,thread_id,internal_date,category_id,category_slug,subject,has_attachment,has_sort_code,sort_code,mentions_bank,mentions_change,mentions_account_specifics,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,attachments,email_text_body
0,186977,1,2.0,186977_t,2021-09-14 10:07:55+00,35,bank-detail-change,RE: Tyronex Bank Account Change,False,False,None,True,True,False,mina.pasaloglou@evo-group.co.uk,mina.pasaloglou@evo-group.co.uk,"{Matt@tyronex.co.uk,Matt.Hales@evo-group.co.uk...",3,None,None,{},None,"Hi Matt, Thank you for your email. Can ..."
1,186977,1,1.0,186977_t,13 September 2021 10:17,35,bank-detail-change,FW: Tyronex Bank Account Change,False,False,None,True,False,False,[Matt@tyronex.co.uk],Matt Tibble,"[Matt.Hales@evo-group.co.uk, Robert.Mitchell@b...",6,None,None,{},None,Alert: There is currently a serious thre...
2,186977,1,0.0,186977_t,13 September 2021 09:49,35,bank-detail-change,Tyronex Bank Account Change,False,False,None,True,True,False,Matt Tibble,Matt Tibble,"[ Matt.Hales@evo-group.co.uk , Robert.Mitchel...",5,None,None,{},None,Hi All Please find attached our new b...
3,282436,0,NaN,None,2022-05-24 11:58:03+00,35,bank-detail-change,Change of bank details,True,False,None,True,True,False,Kirsty.Teather@sewellwallis.co.uk,Kirsty.Teather@sewellwallis.co.uk,{purchase.ledger@evo-group.co.uk},1,{},{},{},Change of Company Bank Details.pdf,Attention: This email originated outside of ou...
4,283345,0,NaN,None,2022-05-26 07:54:24+00,35,bank-detail-change,New Supplier - Bridgehead International,False,False,None,True,True,False,Elaine.Robins@evo-group.co.uk,Elaine.Robins@evo-group.co.uk,"{Lisa.Dyson@evo-group.co.uk,Matt.Hales@evo-gro...",3,{},{Stuart.Bleese@vowwholesale.co.uk},{},None,HI All Please see attached updated bank...
5,284818,0,NaN,None,2022-05-31 08:05:52+00,35,bank-detail-change,URGENT: Bluestem Group - Change of Bank Account,False,False,None,True,True,False,julie.stallard@bluestemgroup.co.uk,julie.stallard@bluestemgroup.co.uk,{purchase.ledger@evo-group.co.uk},1,{},{},{},None,Attention: This email originated outside of ou...
6,287342,0,NaN,None,2022-06-08 11:06:29+00,35,bank-detail-change,Change of Name & Bank Details,False,True,01-07-80,True,True,True,paul.f@pfwlabels.com,paul.f@pfwlabels.com,{paul.f@pfwlabels.com},1,{},{},{},None,Attention: This email originated outside of ou...
7,290334,0,NaN,None,2022-06-16 12:08:12+00,35,bank-detail-change,UAB Fabrica Moderna,False,False,None,False,True,False,Rachel.Parkin@evo-group.co.uk,Rachel.Parkin@evo-group.co.uk,"{jim.bennett@vowwholesale.co.uk,Andrew.Carr@ev...",19,{},{},{},None,"Afternoon all, Please see below a new su..."
8,290391,0,NaN,None,2022-06-16 14:26:34+00,35,bank-detail-change,NEW ADDRESS,False,False,None,False,True,False,Lorraine.Gledhill@statusinternationaluk.co.uk,Lorraine.Gledhill@statusinternationaluk.co.uk,{purchase.ledger@evo-group.co.uk},1,{},{},{},None,Attention: This email originated outside of ou...
9,291518,0,NaN,None,2022-06-20 15:54:21+00,35,bank-detail-change,Important Changes to Invoicing and Billing,False,True,01-08-81,True,True,True,Molly.Bayes@cways.co.uk,Molly.Bayes@cways.co.uk,{accounts@cways.co.uk},1,{},{},{},None,Attention: This email originated outside of ou...


# To read new dataframe

In [42]:
i = 1
for row in new_df.itertuples():
    print("#" * 70, "\n\n")
    print(f'email index: {i}')
    i += 1
    print(f'subject line:\t{row.subject}')
    print(f'received from:\t{row.from_address}')
    print(f'is part of thread:\t{bool(row.is_part_of_thread)}')
    #print(f'num in thread:\t\t{int(row.num_in_thread)}')
    print(f'has attachment:\t\t{row.has_attachment}')
    print(f'has sort code:\t\t{row.has_sort_code}')
    print(f'mentions bank:\t\t{row.mentions_bank}')
    print(f'mentions b details:\t{row.mentions_account_specifics}')
    print(f'mentions change:\t{row.mentions_change}')
    #print(f'sent to:\t{row.sent_to}')
    print(f'sent to {row.sent_to_how_many} people')
    print(f'date sent:\t{row.internal_date}')
    print(f'slug:\t\t{row.category_slug}\n')
    print(row.email_text_body)

###################################################################### 


email index: 1
subject line:	RE: Tyronex Bank Account Change 
received from:	mina.pasaloglou@evo-group.co.uk
is part of thread:	True
has attachment:		False
has sort code:		False
mentions bank:		True
mentions b details:	False
mentions change:	True
sent to 3 people
date sent:	2021-09-14 10:07:55+00
slug:		bank-detail-change

    Hi Matt,    Thank you for your email. Can you also please provide one of the following:    -Void cheque  -Void payment slip  -A bank statement    Thank you,  Mina      
###################################################################### 


email index: 2
subject line:	FW: Tyronex Bank Account Change
received from:	['Matt@tyronex.co.uk']
is part of thread:	True
has attachment:		False
has sort code:		False
mentions bank:		True
mentions b details:	False
mentions change:	False
sent to 6 people
date sent:	13 September 2021 10:17  
slug:		bank-detail-change

      Alert: There is currently a s

In [ ]:
i = 1
for row in bankchange_emails.itertuples():
    print("#" * 70, "\n\n")
    print(i)
    i += 1
    print(f'subject line:\t{row.subject}')
    print(f'received from:\t{row.from_address}')
    print(f'sent to:\t{row.to}')
    print(f'date sent:\t{row.internal_date}')
    print(f'slug:\t\t{row.category_slug}')
    print(row.body)

In [ ]:
# in the bank_deets df look at each email

split_string = "From: "

print(f'there are {len(bankchange_emails)} emails in total.\n')
for line in bankchange_emails.itertuples():
    print("#" * 70, "\n\n")

    if split_string not in line.body:
        print(f'subject line:\t{line.subject}')
        print(f'received from:\t{line.from_address}')
        print(f'sent to:\t{line.to}')
        print(f'date sent:\t{line.internal_date}')
        print(f'slug:\t\t{line.category_slug}')
        print("")
        print(line.body)
        print("\n")
    else:
        split_emails = line.body.split("From: ")
        for i, row in enumerate(split_emails):
            if i > 0:
                before_sent = row.split("Sent: ")[0]
                #print(before_sent,'\n')
                #print(f'From: {this_person}, {this_email}')
                this_email = re.findall(r'<(.*?)>', before_sent)
                row = re.sub(before_sent,"",row)
                if not(this_email):
                    this_person = before_sent
                else:
                    this_person = re.sub(r'<(.*?)>',"",before_sent)
                start_index = row.find('Sent: ') + len('Sent: ')
                end_index = row.find('To: ')
                date_text = row[start_index:end_index]
                row = re.sub(date_text,"",row)
                print(f'Date: {date_text}')

                start_index = row.find('To: ') + len('To: ')
                end_index = row.find('Subject: ')
                to_text = row[start_index:end_index]
                to_emails = re.findall(r'<(.*?)>', to_text)
                row = re.sub(to_text,"",row)
                print(f'To: {to_emails}')
                print(f'emails sent to: {len(to_emails)} people')
                start_index = row.find('Subject: ') + len('Subject: ')
                end_index = row.find('      ')
                this_subject = row[start_index:end_index]
                print(f'subject: {this_subject}')
                row = re.sub('Sent: ',"",row)
                row = re.sub('To: ',"",row)
                row = re.sub('Subject: ',"",row)
                row = re.sub(this_subject,"",row)

                print('\n')
                print(row)
                print('\n')
            elif i == 0:
                print(f'subject line:\t{line.subject}')
                print(f'received from:\t{line.from_address}')
                print(f'sent to:\t{line.to}')
                print(f'date sent:\t{line.internal_date}')
                print(f'slug:\t\t{line.category_slug}')
                print("")
                print(row)
                print('\n')


2 - how many emails mention the keywords Talal suggested?

e.g. "bank" "IBAN" "sort code" 

In [ ]:
contain_values = labelled_emails[labelled_emails['body'].str.contains('bank | IBAN | sort code')]
print(f'length of total dataset: {len(labelled_emails)}')
print(f'length of emails containing bank/IBAN/sort code: {len(contain_values)}')

In [ ]:
contain_values = bankchange_emails[bankchange_emails['body'].str.contains('bank | IBAN | sort code')]
print(f'length of total dataset: {len(bankchange_emails)}')
print(f'length of emails containing bank/IBAN/sort code: {len(contain_values)}')

3 - how many emails include a sort code in the email? 

Like a regex string?

In [ ]:
srt_code_string = r'\b\d{2}-\d{2}-\d{2}\b'


# Look at other categories here if wanted:

In [ ]:
# in the bank_deets df look at each email
for row in labelled_emails[labelled_emails.slug.str.contains("bank-detail-change") == True].itertuples():
    print("#" * 70, "\n\n")
    print(f'subject line: {row.subject}')
    print(f'slug: {row.slug}')
    print("")
    print(row.body)
    print("\n")

# Regex run through 

Find all sort code matches in the emails

In [ ]:
# to do with Regex. Clean first:

# remove phone numbers
# find if it has a sort account
# find if the body of the text contains "change", "new"
# potentially save if hyperlink was present in email?

import re 
import string

def clean_text_re_round1(text):
    '''Clean text in following ways:
    - remove text in square brackets [] and within a html tag <>
    - remove all https links
    - find if sort code included in email
    ? find if the body of the email contains "bank","change","new","sort code"
    '''
    
    text = re.sub('\[.*?\]', '', text) # clean text from within square brackets 
    text = re.sub('\<.*?\>', '', text) # clean text from inside hyperlinks
    text = re.sub('\{.*?\}', '', text) # clean text from inside curly brackets
    
    return text


In [ ]:
# example of some cleaning with this
df['body'][115]

In [ ]:
a = clean_text_re_round1(df['body'][115])
a

In [ ]:
# iteratr
for i,row in df.head(150).iterrows():
    df.loc[i,'clean_body'] = clean_text_re_round1(df.loc[i,'body'])
#     row.clean_body = clean_text_re_round1(row.body)

In [ ]:
# clean_text = []
# for row in df.head(150).itertuples():
#     clean_text.append(clean_text_re_round1(row.body))


In [ ]:
df['clean_body']

In [ ]:
# https://stackoverflow.com/questions/11518035/regular-expression-for-gb-based-and-only-numeric-phone-number
# function to clean all numbers that aren't bank account details
# to do:
# clean phone 
# - has sort code
# - 
def clean_text_re_round2(text):
    '''Clean text in following ways:
    - remove phone numbers?
    - extract sort code numbers
    '''
    #phn_number_string = r'^(((\+44\s?\d{4}|\(?0\d{4}\)?)\s?\d{3}\s?\d{3})|((\+44\s?\d{3}|\(?0\d{3}\)?)\s?\d{3}\s?\d{4})|((\+44\s?\d{2}|\(?0\d{2}\)?)\s?\d{4}\s?\d{4}))(\s?\#(\d{4}|\d{3}))?$'
    srt_account_string = r'^(\d{2}-?\d{2}-?\d{2})$'

    str_account_two = r'bank.account'
    str_account_three = r'sort.code'
    
    
    
    return 

# Regex experimentation

In [ ]:
import re

#srt_account_string = r'\d{2}-?\d{2}-?\d{2}'

#srt_account_string = r'(\d{6}|(\d{2}-){2}\d{2})'

#srt_account_string = r'/^(?!(?:0{6}|00-00-00))(?:\d{6}|\d\d-\d\d-\d\d)$/'

#srt_account_string = r'(?:\d{6}|\d\d-\d\d-\d\d)$/'

srt_account_string = r'^(\d{2}-?\d{2}-?\d{2})$'

#str_account_two = r'bank.account'
#str_account_three = r'sort.code'


In [ ]:
df['sortaccounts_val'] = df['body'].str.findall(srt_account_string).apply(', '.join)

In [ ]:
#df['sortaccounts_val']

In [ ]:
# to make it a boolean column variable
#df['sortaccounts'] = df['body'].str.contains(srt_account_string)
#df['sortaccounts'] = df['body'].str.extract(srt_account_string) # extract the actual values from here
df['sortaccounts'] = df['body'].str.contains(srt_account_string)

num_emails = sum(df['sortaccounts'])

print(f'num emails with sort accounts: {num_emails}')
df['sortaccounts']

In [ ]:
str_account_two = r'bank.account'
str_account_three = r'sort.code'

df['sortaccounts_val'] = df['body'].str.findall(str_account_three).apply(', '.join)

df['sortaccounts'] = df['body'].str.contains(str_account_three)

num_emails = sum(df['sortaccounts'])

print(f'num emails with sort accounts: {num_emails}')
df['sortaccounts']

In [ ]:
for row in df[df["sortaccounts"] == True].head(10).itertuples():
    print("#" * 70, "\n\n")
    print(f'sort account: {row.sortaccounts_val}')
    print(row.subject)
    
    print("")
    print(row.body)
    print("\n")
